In [0]:
# A simple example of using the Salesforce API
# Following the instructions at https://pypi.org/project/simple-salesforce/
# Requires your organisation to have single sign-on (some organisations turn this off, e.g. if you also use Okta)
# To get your Salesforce security token, follow instructions at https://help.salesforce.com/articleView?id=user_security_token.htm&type=5:
# .   - save this is a configuration file, e.g. like:
# [configuration]
# username = YOUR_USERNAME
# password = YOUR_PASSWORD
# token = YOUR_TOKEN

!pip install simple-salesforce

In [0]:
# Load config file (apologies for convoluted method to work with Colab)
from google.colab import files
import pandas as pd
import io

TOKEN_FILE_NAME = 'YOUR_TOKEN_FILE_NAME'

print("Upload token file for Salesforce:")
uploaded = files.upload()

import configparser

config = configparser.ConfigParser()
config.read("config.txt")
username = config["configuration"]["username"]
password = config["configuration"]["password"]
token = config["configuration"]["token"]

In [0]:
# Authenticate with Salesforce
from simple_salesforce import Salesforce

sf = Salesforce(username=username, password=password, security_token=token)

In [0]:
# 1) Simple one-off query, featuring the two types of SOQL joins ('up' and 'down')
# See https://www.sfdc99.com/2013/06/09/example-how-to-write-a-cross-object-soql-query/ and 
# https://www.sfdc99.com/2013/06/24/example-how-to-write-a-cross-object-soql-query-part-2/ for more tips on SOQL joins
# Schema at https://developer.salesforce.com/docs/atlas.en-us.api.meta/api/sforce_api_erd_majors.htm
query = """
select id, name, createddate, account.name, (select contact.name from opportunitycontactroles)
from opportunity
order by createddate asc
limit 10
"""
sf.query(query)

In [0]:
# 2) More complex example for larger data sets: loop through a date range day by day and concatenate results
import datetime

# First create list of dates
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(188)]
date_list.reverse()

In [0]:
# Then loop through list of dates with one query per day
for i in range(0,len(date_list)):
    print(i)
    if i < len(date_list):
        from_date = date_list[i].strftime("%Y-%m-%d") + "T00:00:00.000+0000"
        to_date = date_list[i+1].strftime("%Y-%m-%d") + "T00:00:00.000+0000"
        
        query = """
select leadid, createddate, field, oldvalue, newvalue from leadhistory
where createddate >= """ + from_date + """ and createddate < """ + to_date + """
and field='LeadSource'
order by createddate asc
"""
        response = sf.query_all(query)
        df = pd.DataFrame(response['records'], columns=response['records'][1].keys())
        if i == 0:
            all_df = df
        else:
            all_df = pd.concat([all_df, df], ignore_index=True)

In [0]:
# Output results to csv
all_df.to_csv(r'lead_source_changes_salesforce.csv')